In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  7% |  0% |


In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import os
from torchmetrics import F1Score

In [4]:
torch.__version__

'1.13.0+cu117'

In [29]:
os.chdir('input/data')
os.getcwd()

'/opt/ml/input/data'

In [30]:
!find . -regex ".*\.\_[a-zA-Z0-9._]+" -delete

In [4]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [5]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
dt_train

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [6]:
def get_age_range(age):
    if age < 30:
        return 0
    elif 30 <= age < 60:
        return 1
    else:
        return 2

In [7]:
dt_train['age_range'] = dt_train['age'].apply(lambda x : get_age_range(x))

In [8]:
dt_train

,id,gender,race,age,path,age_range
0,000001,female,Asian,45,000001_female_Asian_45,1
1,000002,female,Asian,52,000002_female_Asian_52,1
2,000004,male,Asian,54,000004_male_Asian_54,1
3,000005,female,Asian,58,000005_female_Asian_58,1
4,000006,female,Asian,59,000006_female_Asian_59,1
...,...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19,0
2696,006955,male,Asian,19,006955_male_Asian_19,0
2697,006956,male,Asian,19,006956_male_Asian_19,0
2698,006957,male,Asian,20,006957_male_Asian_20,0


In [9]:
train_idx, valid_idx = train_test_split(np.arange(len(dt_train)),
                                       test_size=0.2,
                                       shuffle=True,
                                       stratify=dt_train['age_range'])

In [10]:
train_image = []
train_label = []

for idx in train_idx:
    path = dt_train.iloc[idx]['path']
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        train_image.append(train_image_path+path+'/'+file_name)
        train_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))                            

In [11]:
valid_image = []
valid_label = []

for idx in valid_idx:
    path = dt_train.iloc[idx]['path']
    for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
        valid_image.append(train_image_path+path+'/'+file_name)
        valid_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))              

In [12]:
def onehot_enc(x):
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

In [13]:
train_data = pd.Series(train_image)
train_label = pd.Series(train_label)

valid_data = pd.Series(valid_image)
valid_label = pd.Series(valid_label)

In [14]:
class Dataset_Mask(Dataset):
    def __init__(self, data, label, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = data
        self.label = label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[70:420, 17:367]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [15]:
mask_train_set = Dataset_Mask(data=train_data, label=train_label, transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))
mask_val_set = Dataset_Mask(data=valid_data, label=valid_label, transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

In [16]:
batch_size = 32
train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [19]:
SwinT = torchvision.models.swin_v2_s(weights=torchvision.models.Swin_V2_S_Weights.IMAGENET1K_V1)
SwinT

Downloading: "https://download.pytorch.org/models/swin_v2_s-637d8ceb.pth" to /opt/ml/.cache/torch/hub/checkpoints/swin_v2_s-637d8ceb.pth


SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='

In [20]:
import math

for param in SwinT.parameters():
    param.requires_grad = False

n_inputs = SwinT.head.in_features
class_num = 18
SwinT.head = nn.Linear(in_features=n_inputs, out_features=class_num, bias=True)
nn.init.xavier_uniform_(SwinT.head.weight)
stdv = 1. / math.sqrt(SwinT.head.weight.size(1))
SwinT.head.bias.data.uniform_(-stdv, stdv)
print(SwinT.head)

Linear(in_features=768, out_features=18, bias=True)


In [26]:
print('필요 입력 채널 개수', SwinT.features[0][0].weight.shape[1])
print('네트워크 출력 채널 개수', SwinT.head.weight.shape[0])

필요 입력 채널 개수 3
네트워크 출력 채널 개수 18


In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

model = SwinT.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 50

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

using cuda:0


In [28]:
best_val_acc = 0
best_val_loss = np.inf
patience = 5
cur_count = 0

f1 = F1Score(num_classes=class_num, average='macro').to(device)
best_f1_score = 0

for epoch in range(NUM_EPOCH):
    model.train()
    loss_value = 0
    matches = 0
    for train_batch in tqdm(train_dataloader_mask):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            torch.save(model, '/opt/ml/checkpoint/swin_v2_s/checkpoint_ep_%d.pt'% epoch)
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
    train_loss = loss_value / len(train_dataloader_mask)
    train_acc = matches / len(mask_train_set)
    
    print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.5f}, training accuracy {train_acc:.5f}")
    
    with torch.no_grad():
        model.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in tqdm(val_dataloader_mask):
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)
        
        f1_score = f1(outs, labels)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            cur_count = 0
            torch.save(model, '/opt/ml/checkpoint/swin_transfrmer_t_2/checkpoint_best.pt')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
        
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            cur_count = 0
            torch.save(model, '/opt/ml/checkpoint/swin_v2_s/checkpoint_best.pt')
        else:
            cur_count += 1
            if cur_count >= patience:
                print("Early Stopping!")
                break
                
        print(f"[val] acc : {val_acc:.5f}, loss : {val_loss:.5f}")
        print(f"best acc : {best_val_acc:.5f}, best loss : {best_val_loss:.5f}")


epoch[0/50] training loss 2.66538, training accuracy 0.15172



[val] acc : 0.20688, loss : 2.41405
best acc : 0.20688, best loss : 2.41405



epoch[1/50] training loss 2.33554, training accuracy 0.19874



[val] acc : 0.22037, loss : 2.21528
best acc : 0.22037, best loss : 2.21528



epoch[2/50] training loss 2.17641, training accuracy 0.23181


...


[val] acc : 0.71296, loss : 0.95124
best acc : 0.71323, best loss : 0.95124



epoch[48/50] training loss 0.92167, training accuracy 0.72765



[val] acc : 0.71455, loss : 0.94300
best acc : 0.71455, best loss : 0.94300



epoch[49/50] training loss 0.91043, training accuracy 0.73128



[val] acc : 0.71852, loss : 0.93582
best acc : 0.71852, best loss : 0.93582


In [17]:
import timm
from timm.loss import LabelSmoothingCrossEntropy

model = torch.load('/opt/ml/checkpoint/swin_v2_s/80+_checkpoint_new_best.pt')

for param in model.parameters():
    param.requires_grad = True

In [18]:
np.set_printoptions(precision=3)
n_param = 0
for p_idx, (param_name, param) in enumerate(model.named_parameters()):
    if param.requires_grad:
        param_numpy = param.detach().cpu().numpy()
        n_param += len(param_numpy.reshape(-1))
        print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
        print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
print ("Total number of parameters:[%s]."%(format(n_param,',d')))

[0] name:[features.0.0.weight] shape:[(96, 3, 4, 4)].
    val:[ 0.022 -0.027 -0.023 -0.03   0.013]
[1] name:[features.0.0.bias] shape:[(96,)].
    val:[ 0.158 -0.164  0.002 -0.001  0.056]
[2] name:[features.0.2.weight] shape:[(96,)].
    val:[1.37  1.187 4.375 5.708 4.715]
[3] name:[features.0.2.bias] shape:[(96,)].
    val:[-1.55   1.612  0.994  1.179 -0.708]
[4] name:[features.1.0.norm1.weight] shape:[(96,)].
    val:[1.005 0.475 1.347 1.499 1.295]
[5] name:[features.1.0.norm1.bias] shape:[(96,)].
    val:[-0.367 -0.722 -0.034 -0.099  0.043]
[6] name:[features.1.0.attn.logit_scale] shape:[(3, 1, 1)].
    val:[1.482 0.409 2.019]
[7] name:[features.1.0.attn.qkv.weight] shape:[(288, 96)].
    val:[-0.014 -0.028 -0.07   0.057 -0.032]
[8] name:[features.1.0.attn.qkv.bias] shape:[(288,)].
    val:[ 0.237  0.174 -0.218 -0.373 -0.035]
[9] name:[features.1.0.attn.proj.weight] shape:[(96, 96)].
    val:[-0.031  0.012 -0.019 -0.017  0.039]
[10] name:[features.1.0.attn.proj.bias] shape:[(96,)].


In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

model = model.to(device)

LEARNING_RATE = 0.001
NUM_EPOCH = 200
class_num = 18

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

using cuda:0


In [20]:
best_val_acc = 0
best_val_loss = np.inf
# patience = 5
# cur_count = 0

f1 = F1Score(num_classes=class_num, average='macro').to(device)
best_f1_score = 0

for epoch in range(NUM_EPOCH):
    model.train()
    loss_value = 0
    matches = 0
    for train_batch in tqdm(train_dataloader_mask):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            torch.save(model, f'/opt/ml/checkpoint/swin_v2_s/nonfreeze_checkpoint_ep_{epoch}.pt')
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
    train_loss = loss_value / len(train_dataloader_mask)
    train_acc = matches / len(mask_train_set)
    
    scheduler.step()
    
    print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.5f}, training accuracy {train_acc:.5f}")
    
    with torch.no_grad():
        model.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in tqdm(val_dataloader_mask):
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)
        
        f1_score = f1(outs, labels)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
#             cur_count = 0
#             torch.save(model, '/opt/ml/checkpoint/swin_v2_s/nonfreeze_checkpoint_new_best.pt')                     
#         else:
#             cur_count += 1
#             if cur_count >= patience:
#                 print("Early Stopping!")
#                 break
                
        if val_acc > best_val_acc:
            best_val_acc = val_acc
        
        if f1_score > best_f1_score:
            best_f1_score = f1_score

                
        print(f"[val] acc : {val_acc:.5f}, loss : {val_loss:.5f}, f1 score : {f1_score:.5f}")
        print(f"best acc : {best_val_acc:.5f}, best loss : {best_val_loss:.5f}, best f1 : {best_f1_score:.5f}")


epoch[0/200] training loss 2.68546, training accuracy 0.18657



[val] acc : 0.22751, loss : 2.47286, f1 score : 0.22222
best acc : 0.22751, best loss : 2.47286, best f1 : 0.22222



epoch[1/200] training loss 2.43199, training accuracy 0.20013



[val] acc : 0.27328, loss : 2.29444, f1 score : 0.00000
best acc : 0.27328, best loss : 2.29444, best f1 : 0.22222



epoch[2/200] training loss 2.33824, training accuracy 0.23896


...


epoch[77/200] training loss 2.36458, training accuracy 0.21171



[val] acc : 0.22751, loss : 2.34891, f1 score : 0.00000
best acc : 0.27328, best loss : 2.29444, best f1 : 1.00000



epoch[78/200] training loss 2.36445, training accuracy 0.21323



[val] acc : 0.22751, loss : 2.34373, f1 score : 0.22222
best acc : 0.27328, best loss : 2.29444, best f1 : 1.00000


KeyboardInterrupt: 

In [20]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

best_model = torch.load('/opt/ml/checkpoint/swin_v2_s/new_checkpoint_ep_10.pt')
submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [21]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [22]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            img = img[70:420, 17:367]
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [23]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToTensor()
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = best_model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission_swin_v2_s_ep81.csv'), index=False)
print('test inference is done!')

test inference is done!


In [ ]:
/opt/ml/checkpoint/swin_transfrmer_t_2/